## Heap Management Module

In [31]:
import nbimporter; nbimporter.options["only_defs"] = False
from ST import *
from SC import *

Each `Node` store the value of start adr of the allocated object on the heap, `nxt` points to the nxt Node, `pid` store the address of the pointer pointing to object in WebAssembly at `adr`. `size` is the size of the Node and `allocated` is to check whether an object is allocated or deallocated in WebAssembly at `adr`

In [32]:
class Node:
    def __init__(self, data = None, nxt=None, allocated=False, size = 0, adr = None, p_adr = None):
        self.adr = adr # address of the object on the heap
        self.nxt = nxt # point to the next block in the Linked-list
        self.allocated = allocated # True, if the memory block is allocated at this address; False, when the allocated object is deallocated at this address
        self.size = size # size of the allocated object
        self.pid = p_adr # the address of the pointer points to the allocated object in WebAssembly Memory
    
    def get_nxt(self):
        return self.nxt
    
    def set_nxt(self,data): # data should be a Node object
        if type(data) != Node and data != None:
            print("error: next should be Node")
        else:
            self.nxt = data
            
    def is_alloc(self): #check whether is allocated or deallocated
        return self.allocated

    def getSize(self): # return size of the Node
        return self.size
    
    def free(self): # used when dispose(p) is called
        self.allocated = False
        self.pid = None
    
    def __str__(self): # for showing the information of the Node
        msg = 'adr is '+ str(self.adr)+', size is ' + str(self.size) + ', is_alloc: ' + str(self.allocated) +', nxt: \n' + str(self.nxt) 
        return msg

`Linked-list` is a heap manager, which manage which address on the WebAssembly Memory is availble for the new allocated object.

In [34]:
class LinkedList:
    def __init__(self,start=None,adr = None):
        self.start = start
        self.startAdr = adr # the adr where the heap start
        self.idList = []
    
    # allocate method
    # using first fit algorithm for finding the target block to allocate
    # p is a pointer variable
    # adr is the address of p in WebAssembly Memory
    def allocate(self,p,adr):#using first fit
        if type(p.tp) != Pointer:
            print("error: shoule be pointer type")
            return
        if adr in self.idList: # test for reallocate the same object
            target = self.search(adr)
            return target.adr , True # return the adr of the Node
        current_block = self.start #start at the first block of the linked-list, search for a block which is None or a block has enough space for the allocated object
        if current_block != None:
            pre_block = None
            while current_block != None: 
                # handle first fit problem
                # current block is not allocated and has enough space for p, then current block is the block to allocate p
                # if the current block does not have enough space for p, then go the next block
                if current_block.is_alloc() == False and current_block.getSize() >= p.tp.base.size: 
                    break
                else:
                    pre_block = current_block
                    current_block = current_block.get_nxt()
            if current_block != None: # means the available block is on the LinkedList 
                if current_block.getSize() == p.tp.base.size: #if the available block size is exact same as the size of p
                    current_block.allocated = True
                    current_block.size = p.tp.base.size
                    current_block.pid = adr
                    if p.tp.base in (Bool, Int):
                        current_block.value = None
                    else:
                        current_block.value = [None]*int(p.tp.base.size/4)
                    self.idList.append(adr)
                    return current_block.adr
                else: # the size of p is smalled than the available block, split the available block into two parts
                    current_block_size = current_block.getSize()
                    adr_1 = current_block.adr
                    adr_2 = current_block.adr + p.tp.base.size
                    new_node_with_data = Node(allocated=True,size=p.tp.base.size,adr = adr_1, p_adr = adr) # one part for p
                    new_node_without_data = Node(size=current_block_size - p.tp.base.size,adr = adr_2) # the other is empty
                    if pre_block != None: 
                        pre_block.set_nxt(new_node_with_data)
                    else: # means the available block is at the beginning of the LinkedList
                        self.start = new_node_with_data
                    new_node_with_data.set_nxt(new_node_without_data)
                    new_node_without_data.set_nxt(current_block.get_nxt())
                    self.idList.append(adr)
                    return adr_1
            else: # means the available block is not on the LinkedList, add new block is added to the end of the LinkedList
                a = pre_block.adr + pre_block.size
                new_node = Node(allocated=True,size=p.tp.base.size,adr = a,p_adr = adr)
                pre_block.set_nxt(new_node)
                self.idList.append(adr)
                return a
        else: # linked-list is empty
            new_node = Node(allocated=True,size=p.tp.base.size,adr = self.startAdr,p_adr = adr)
            self.start = new_node
            self.idList.append(adr)
            return new_node.adr
    
    # deallocate method
    # deallocating an object on the LinkedList based on the given adr
    # adr : the address of the pointer var in webAssembly Memory
    def deallocate(self,adr):
        target = self.search(adr)
        if target == None:
            print("error: not allocated.")
            return False
        self.idList.remove(target.pid)
        target.free() # free the memory
        return True
    
    # search for the block based on the address of the pointer in the webAssembly Memory
    def search(self,adr): 
        target = self.start
        while True: # loop until get the target or reach to the end of the Linked-List
            if target == None:
                return None
            #find the target by checking the name of the var
            if not target.is_alloc():
                target = target.get_nxt()
            elif adr == target.pid:
                return target
            else:
                target = target.get_nxt()